In [1]:
import numpy as np
from qibo.symbols import Z, X
from qibo.hamiltonians import SymbolicHamiltonian
from qibo import models, Circuit, gates
import networkx as nx

def H(distance_matrix):
    num_cities = distance_matrix.shape[0]
    form = 0
    for u in range(num_cities):
        for v in range(u+1, num_cities):
            form += 1 - distance_matrix[u, v]*Z(u)*Z(v)
    ham = SymbolicHamiltonian(0.5*form)
    return ham


def mixer(l):
    form = sum(X(i) for i in range(l))
    return SymbolicHamiltonian(form)
    
    
def construct_graph(edges):
    """
    Construct graph and adjacency matrix given edges list.
    
    Args: 
        edges: edges.
    
    Return:
        graph, adjacency matrix.
    """
    # Create a graph based on the specified topology
    G = nx.Graph()

    G.add_edges_from(edges)

    num_nodes = G.number_of_nodes()
    num_edges = len(G.edges)

    weights = np.random.uniform(0, 20, num_edges)

    # Initialize adjacency matrix with high weights
    high_weight = 0
    adjacency_matrix = np.full((num_nodes, num_nodes), high_weight)

    # Populate the adjacency matrix with weights from the external list
    for idx, (i, j) in enumerate(G.edges()):
        adjacency_matrix[i-1, j-1] = weights[idx]  # Use weights from the external list
        adjacency_matrix[j-1, i-1] = weights[idx]  # Undirected graph: set both (i,j) and (j,i)

    # Optionally set the diagonal to 0 (self-loops with zero weight)
    np.fill_diagonal(adjacency_matrix, 0)

    return G, adjacency_matrix

In [2]:
edges = [
    # (0, 1), (0, 2), (0, 3), (0, 6), (0, 8), (0, 10), (0, 11), (0, 12),
    (1, 2), (1, 4), # (1, 7), (1, 9), (1, 10),
    (2, 3), (2, 4),
    (3, 4),
    # (4, 5), (4, 7),
    # (5, 6), (5, 7),
    # (6, 8), (6, 7),
    # (7, 8), (7, 9),
    # (8, 9), (8, 10),
    # (9, 10),
    # (10, 11),
    # (11, 12)
]
G, matrix = construct_graph(edges)

In [3]:
# matrix = np.array([[0, 0.1, 1], [2, 0, 0.2],[0.3, 3, 0]])
num_cities = 4
qaoa = models.QAOA(H(matrix), mixer=mixer(num_cities))

circuit = Circuit(num_cities)
circuit.add(gates.H(i) for i in range(num_cities))
initial_state = circuit().state()

[Qibo 0.2.11|INFO|2024-10-24 18:49:00]: Using qibojit (numba) backend on /CPU:0


In [7]:
nlayers = 4
best_energy, final_parameters, extra = qaoa.minimize(0.1 * np.random.random(2*nlayers),initial_state=initial_state)

In [8]:
best_energy

-16.43370729617633

In [10]:
H(matrix).eigenvalues()

[Qibo 0.2.11|WARNING|2024-10-24 18:51:34]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


array([-19., -19.,  -5.,  -5.,  -1.,  -1.,   2.,   2.,   5.,   5.,   6.,
         6.,  16.,  16.,  20.,  20.])

In [11]:
circuit = Circuit(num_cities)
circuit.add(gates.M(i) for i in range(num_cities))
freqs = circuit(initial_state = qaoa.execute(), nshots=10000).frequencies()

In [12]:
freqs

Counter({'0000': 4344,
         '1111': 4284,
         '1011': 320,
         '0100': 278,
         '1000': 156,
         '0010': 151,
         '1101': 149,
         '0111': 145,
         '0011': 45,
         '1100': 44,
         '0101': 23,
         '0110': 22,
         '1010': 22,
         '1001': 16,
         '0001': 1})